# 2. Script for obtaining word frequencies in comment corpora

## Set Up

In [1]:
import pandas as pd

In [2]:
path = "C:\\Users\\nicol\\Documents\\2021-2022\\QMSS\\Spring'22\\QMSS 5999 Thesis\\data\\"

# Import comment data
reddit_raw = pd.read_csv(path+'comments_reddit_all.csv', index_col=0, dtype={'comment_id': 'str', 'text': 'str'})
fb_raw = pd.read_csv(path+'comments_fb_all.csv', index_col=0, dtype={'comment_id': 'str', 'text': 'str'})
fb_raw = fb_raw[['comment_id', 'text']]

In [3]:
# Concatenate both FB and Reddit data into one DataFrame
all_raw = pd.concat([reddit_raw, fb_raw], ignore_index=True)

In [4]:
# Inspect DataFrame and get dimensions
display(all_raw.head())
display(all_raw.tail())
display(all_raw.shape)

,comment_id,text
0,gww5drd,ALL MY PLANS ARE GONE NOOOOOOOO
1,gww5jfz,&gt; This means people will no longer be allow...
2,gww68ne,phase 2 lai liao wah shag
3,gww6icr,Government fucks up and takes it out on the po...
4,gww6luf,Gyms also closed till 30th. PepeHands. I just ...


,comment_id,text
15714,10158186215552115,Trying very hard to push that all singaporeans...
15715,10158186213932115,Omg. This is real bad. And really bad foresigh...
15716,10158186213787115,"Endemic ma, didnt they know this?"
15717,10158186211122115,"If healthcare system is overstretched, why you..."
15718,10158186210967115,Overwhelmed by antivaxxers


(15719, 2)

## Preprocessing text

### Clean text

In [5]:
# Function for cleaning text
def clean_text_freq(text):
    import re
    import html
    text1 = html.unescape(text) # remove HTML encoded characters
    text2 = re.sub(r'â€™', '\'', text1) # replce â€™ with apostrophe
    text3 = re.sub(r'\n', '', text2) # remove excess line breaks between links
    text4 = re.sub(r'\n', ' ', text3) # replace remaining line breaks with spaces
    text5 = re.sub(r'#\S+', '', text4) # remove hashtags
    text6 = re.sub(r'http\S+', '', text5) # remove links starting with http
    text7 = re.sub(r'\S+\.com\S+', '', text6) # remove links not starting with http
    text8 = re.sub(r'[^A-Za-z]+', ' ', text7).strip().lower() # remove punctuations and digits 
    return(text8)

In [6]:
# Apply function to DataFrame
all_raw['text_clean_freq'] = all_raw.text.apply(clean_text_freq)

In [7]:
# Inspect DataFrame
all_raw.head()

,comment_id,text,text_clean_freq
0,gww5drd,ALL MY PLANS ARE GONE NOOOOOOOO,all my plans are gone noooooooo
1,gww5jfz,&gt; This means people will no longer be allow...,this means people will no longer be allowed to...
2,gww68ne,phase 2 lai liao wah shag,phase lai liao wah shag
3,gww6icr,Government fucks up and takes it out on the po...,government fucks up and takes it out on the po...
4,gww6luf,Gyms also closed till 30th. PepeHands. I just ...,gyms also closed till th pepehands i just want...


### Remove stop words

In [8]:
# Function for removing stop words
def remove_sw(text):
    from nltk.corpus import stopwords
    sw = set(stopwords.words('English'))
    text_filtered = [word for word in text.split() if not word in sw] 
    text_filtered = ' '.join(text_filtered) 
    return(text_filtered)

In [9]:
# Apply function to DataFrame
all_raw['text_no_sw'] = all_raw.text_clean_freq.apply(remove_sw)

In [10]:
# Inspect DataFrame
all_raw.head()

,comment_id,text,text_clean_freq,text_no_sw
0,gww5drd,ALL MY PLANS ARE GONE NOOOOOOOO,all my plans are gone noooooooo,plans gone noooooooo
1,gww5jfz,&gt; This means people will no longer be allow...,this means people will no longer be allowed to...,means people longer allowed digitally check sc...
2,gww68ne,phase 2 lai liao wah shag,phase lai liao wah shag,phase lai liao wah shag
3,gww6icr,Government fucks up and takes it out on the po...,government fucks up and takes it out on the po...,government fucks takes population changes work...
4,gww6luf,Gyms also closed till 30th. PepeHands. I just ...,gyms also closed till th pepehands i just want...,gyms also closed till th pepehands want rock c...


### Tokenize comment corpus

In [11]:
# Transform text in DataFrame to a single string
all_raw_string = ' '.join(all_raw['text_no_sw']) 

In [13]:
# Tokenise the whole corpus
from nltk.tokenize import word_tokenize
tokens_all = word_tokenize(all_raw_string)
print(len(tokens_all))

209387


In [44]:
# Obtain tokens not in NLTK's English dictionary (to identify non-English words that need to be parsed/replaced)
import nltk
from nltk.corpus import words
from nltk.corpus import wordnet

nltk_words = words.words()
tokens_no_eng = [tok for tok in tokens_all if not tok in nltk_words if not wordnet.synsets(tok)]

## Obtain top 1000 tokens by frequency

### Whole corpus

In [14]:
# Obtain top 1000 tokens from the whole corpus
from nltk.probability import FreqDist
fdist_all = FreqDist(tokens_all)
tokens_all_top1000 = fdist_all.most_common(1000)

In [46]:
# Inspect top 20 
tokens_all_top1000[0:20]

[('people', 2230),
 ('covid', 1503),
 ('like', 1417),
 ('still', 1249),
 ('cases', 1188),
 ('vaccinated', 1165),
 ('get', 1159),
 ('go', 1107),
 ('one', 991),
 ('even', 961),
 ('time', 888),
 ('us', 853),
 ('need', 842),
 ('think', 828),
 ('back', 824),
 ('singapore', 822),
 ('also', 810),
 ('going', 767),
 ('open', 731),
 ('government', 698)]

In [15]:
# Inspect bottom 20
tokens_all_top1000[-20:]

[('walk', 40),
 ('leaving', 40),
 ('add', 40),
 ('brain', 40),
 ('shop', 40),
 ('city', 40),
 ('celebrate', 39),
 ('outdoor', 39),
 ('achieve', 39),
 ('leadership', 39),
 ('thats', 39),
 ('panic', 39),
 ('nz', 39),
 ('steps', 39),
 ('protected', 39),
 ('alot', 39),
 ('mistake', 39),
 ('efforts', 39),
 ('manage', 39),
 ('aug', 39)]

In [47]:
# Convert to DataFrame and inspect
tokens_all_top1000_df = pd.DataFrame(tokens_all_top1000, columns=['token', 'freq'])
tokens_all_top1000_df.head()

,token,freq
0,people,2230
1,covid,1503
2,like,1417
3,still,1249
4,cases,1188


In [48]:
# Export as CSV
tokens_all_top1000_df.to_csv(path+'comments_all_tokens_top1000.csv', encoding='utf-8-sig')

### Non-English corpus

In [49]:
# Obtain top 1000 non-English tokens
from nltk.probability import FreqDist
fdist_no_eng = FreqDist(tokens_no_eng)
tokens_no_eng_top1000 = fdist_no_eng.most_common(1000)

In [50]:
# Inspect top 20 
tokens_no_eng_top1000[0:20]

[('cb', 453),
 ('ktv', 382),
 ('govt', 372),
 ('lol', 297),
 ('ppl', 238),
 ('gov', 223),
 ('etc', 220),
 ('wfh', 166),
 ('hbl', 155),
 ('others', 143),
 ('moh', 138),
 ('pls', 138),
 ('mrt', 134),
 ('mtf', 112),
 ('ndp', 103),
 ('hari', 102),
 ('vax', 100),
 ('mmtf', 99),
 ('oyk', 99),
 ('reddit', 85)]

These non-English tokens will guide me in constructing a dictionary to replace these words with their meaning, to allow the sentiment analysis algorithms to better parse the text and to improve the results. 

Additionally, this list helps me to decide which tokens should be kept:
- For instance, 'cb' is a top token but it has multiple meanings in the corpora as it refers to either a swear word ('cb' short for 'chee bye' which is equivalent in vulgarity to the F word), or the Circuit Breaker lockdown restrictions in April 2020 (which the government and citizens have commonly abbreviated into 'CB'). Given the confusion, I will eliminate this specifc token from the list. I will only retain 'cb' if it is part of another token like 'ccb' which definitely refers to the swear phrase ('chao chee bye') and not the restrictions. 
- I will also not replace tokens which do not have a concise explanation/replacement in the English language, or tokens that are proper nouns.
- Internet slang abbreviations e.g. lol, lmao, etc. will not be replaced as they can be captured by VADER.
- Tokens with frequency <= 50 will not be replaced.

In [51]:
# Convert to DataFrame and export as CSV
tokens_no_eng_top1000_df = pd.DataFrame(tokens_no_eng_top1000, columns=['token', 'freq'])
tokens_no_eng_top1000_df.to_csv(path+'comments_all_tokens_no_eng_top1000.csv', encoding='utf-8-sig')